# 0. helper funcs & params

In [6]:
import threading,time
from tqdm import tqdm
import pandas as pd


def multi(func,nums):
    results = [None] * len(nums)

    def compute_and_store_result(i, x):
        results[i] = func(x)

    threads = []
    for i, x in enumerate(nums):
        thread = threading.Thread(target=compute_and_store_result, args=(i, x))
        threads.append(thread)
        thread.start()

    for thread in tqdm(threads, desc="Progress"):
        thread.join()

    return results


import os

def get_all_files(folder_path):
    file_paths = []
    for root, directories, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            file_paths.append(file_path)
    return file_paths


task_dir=os.getcwd()
max_qnum=29

idxs=[['{0}'.format(i) for i in range(1,max_qnum+1)],
      ['(a)','(b)','(c)','(d)','(e)','(f)','(g)','(h)'],
      ['(i)','(ii)','(iii)','(iv)','(v)','(vi)','(vii)']]

print(task_dir)

d:\server\230621片题_数学\Redo2


# 1.get page images

In [7]:
# detect margin
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

def plot_numbers(numbers):
    # Create x-axis values from 0 to (number of elements - 1)
    x = range(len(numbers))
    
    # Plot the numbers
    plt.plot(x, numbers)
    
    # Add labels and title
    plt.xlabel('Index')
    plt.ylabel('Number')
    plt.title('Plot of Numbers')
    
    # Display the plot
    plt.show()

def convert_pixels_below_threshold(image, threshold):
    # Convert image to numpy array
    image_array = np.array(image)

    # Apply thresholding operation
    image_array[image_array < threshold] = 255

    # Convert numpy array back to image
    result_image = Image.fromarray(image_array)

    return result_image

def detect_peaks(data,debug=False,max_mar=1/8):
    peaks = []
    n = len(data)
    
    # Check if the list has at least three elements
    if n < 3:
        return peaks
    
    # Check the first element
    if data[0] >= data[1]:
        is_peak = True
    else:
        is_peak = False
    
    start_index = 0
    
    # Iterate over the remaining elements
    for i in range(1, n - 1):
        # Check if the current element is greater than or equal to its neighbors
        if data[i] >= data[i - 1] and data[i] >= data[i + 1]:
            if not is_peak:
                start_index = i
                is_peak = True
        else:
            if is_peak:
                end_index = i - 1
                peak_value = max(data[start_index:end_index+1])
                peak_width = end_index - start_index + 1
                peaks.append((start_index, peak_width, peak_value))
                is_peak = False
    
    # Check the last element
    if is_peak:
        end_index = n - 1
        peak_value = max(data[start_index:end_index+1])
        peak_width = end_index - start_index + 1
        peaks.append((start_index, peak_width, peak_value))

    if debug:
        plot_numbers(data)
    peaks = sorted(peaks,key=lambda x: -x[2])
    ver_cut=[None,None]
    for peak in peaks:
        
        position, width, value = peak
        if debug:
            print(f"Peak detected at position {position}, width {width}, value {value}")
        if position<max_mar*len(data) and ver_cut[0] == None:
            ver_cut[0]=position+width
        
        if position>(1-max_mar)*len(data) and ver_cut[1] == None:
            ver_cut[1]=position-width

        if None not in ver_cut:
            break

        
    #print('cut:',ver_cut)

    
    return ver_cut


from pdf2image import convert_from_path
from PIL import ImageDraw
def get_margin(image,debug=False):

    #display(image)
    image_cover=convert_pixels_below_threshold(image,100)
    #display(image_cover)
    img_data=np.array(image_cover)
    img_data_leftsum=255-img_data.mean(axis=0)
    img_data_downsum=255-img_data.mean(axis=1)
    
    #plot_numbers(img_data_leftsum)
    ver_cut=detect_peaks(img_data_leftsum,debug=debug)
    

    #plot_numbers(img_data_downsum)
    hor_cut=detect_peaks(img_data_downsum,debug=debug)
    if debug:
        print('ver_cut:',ver_cut)
        print('hor_cut:',hor_cut)
    W,H=image.size
    draw=ImageDraw.Draw(image)
    # draw ver cut
    for c in ver_cut:
        draw.line((c,0,c,H),fill='red')

    for c in hor_cut:
        draw.line((0,c,W,c),fill='red')
    if debug:
        display(image)
    left=ver_cut[0]
    right=ver_cut[1]
    top=hor_cut[0]
    buttom=hor_cut[1]
    return [left,top,right,buttom]
    #for c in hor_cut:





def test():
    fp_dir=os.path.join(task_dir,'0-input')
    #fp=random.choice(fps)
    #pdf_fps=[fp]

    pdf_fps=[fp for fp in get_all_files(fp_dir) if fp.endswith('.pdf')]
    
    for fp in pdf_fps:
        pgs=convert_from_path(fp,grayscale=True,first_page=2,last_page=2)
        print(fp)
        get_margin(pgs[0],debug=True)

#test()

In [14]:
import random,PyPDF2
from pdf2image import convert_from_path

pdf_dir=os.path.join(task_dir,'0-input')
pdf_fns=os.listdir(pdf_dir)
pdf_fps=[os.path.join(pdf_dir,fn) for fn in pdf_fns if fn.endswith('.pdf')]
pages_out_dir=os.path.join(task_dir,'1-pdf2images')


def fp2imgs_crop(fp,remove_head=True,end_code='total for paper',debug=False):
    
    imgs=convert_from_path(fp,grayscale=True)
    #print(len(imgs))
    if end_code != '':
        with open(fp, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            for page_num in list(range(len(reader.pages)))[::-1]:
                page = reader.pages[page_num]
                text = page.extract_text()
                #if len(text.replace('....',''))<50:
                #    display(imgs[page_num])
                if debug:
                        print('page_num:',page_num)
                        display(imgs[page_num])
                        print(text)
                if end_code in text.lower():
                    imgs=imgs[:page_num+1]
                    break

    if remove_head:
        imgs=imgs[1:]
    precrops=get_margin(imgs[0])
    for i,page in enumerate(imgs):
        # precrop
        imgs[i]=page.crop(tuple(precrops))

    # save
    fn=fp.split('\\')[-1].split('.')[0]
    #print(fn)
    cur_pdf_imgs_dir=os.path.join(pages_out_dir,fn)
    #print(cur_pdf_imgs_dir)
    if not os.path.exists(cur_pdf_imgs_dir):
        os.makedirs(cur_pdf_imgs_dir)

    pages = imgs

    for ii,page in enumerate(pages):
        image_fn=fn+'_'+"page{:03d}".format(ii+1)+'.png'
        with open(os.path.join(cur_pdf_imgs_dir,image_fn), 'wb') as f:
            page.save(f,format='PNG')
            f.close()

    return len(imgs)

def test():
    fp=random.choice(pdf_fps)
    #fp=r'D:\server\230621片题_数学\0-input\wdm11_01_que_20230126.pdf'
    print(fp)
    num_imgs=fp2imgs_crop(fp,debug=True)
    print('#imgs:',num_imgs)
    #display(imgs[0])
    #display(imgs[-1])

#test()


In [15]:
page_lens=multi(fp2imgs_crop,pdf_fps)
print(page_lens)
print('#paper:',len(page_lens),'#page:',sum(page_lens),'avg page per paper:',sum(page_lens)/len(page_lens))

Progress: 100%|██████████| 1/1 [00:02<00:00,  2.19s/it]

[26]
#paper: 1 #page: 26 avg page per paper: 26.0


# 2.delete appendix pages

In [16]:
import shutil,os
pages_out_dir=os.path.join(task_dir,'1-pdf2images')
# destination folder path
pure_img_folder = os.path.join(task_dir,'2-pure_questions_images','question_images')
deleted_dir=os.path.join(task_dir,'2-pure_questions_images','deleted_images')
if not os.path.exists(deleted_dir):
    os.makedirs(deleted_dir)
# copy the folder and its contents
shutil.copytree(pages_out_dir, pure_img_folder,dirs_exist_ok=True)


'd:\\server\\230621片题_数学\\Redo2\\2-pure_questions_images\\question_images'

In [17]:
from pytesseract import Output
import pytesseract
import pandas as pd
pytesseract.pytesseract.tesseract_cmd = r'E:\Tesseract\tesseract.exe'
from PIL import Image,ImageFont,ImageDraw, ImageFilter

def delete_no_text(fp,debug=False,min_cnum=54):
    im=Image.open(fp)
    txt=pytesseract.image_to_string(im)
    if debug:
            print(fp.split('\\')[-1], len(txt),txt)
            display(im)
    if len(txt)<min_cnum:
        fn=fp.split('\\')[-1]
        if not debug:
            shutil.move(dst=os.path.join(deleted_dir,fn),src=fp)
        
        return True
        #print('deleted '+fn)
    else:
         return False
    
# test
def test():
    deleted=[]
    fps=get_all_files(r'D:\server\230621数学临时片题\1-pdf2images\P1 2022 1月 问卷')
    for fp in fps:
        if delete_no_text(fp,debug=True):
            deleted.append(fp)
    print(len(deleted))

#test()

In [18]:
fps=get_all_files(pure_img_folder)
dels=multi(delete_no_text,fps)


Progress: 100%|██████████| 26/26 [00:01<00:00, 25.92it/s]


In [19]:
print('deleted {0} from {1}, left {2}'.format(len([d for d in dels if d]),len(dels),len([d for d in dels if not d])))

deleted 9 from 26, left 17


## now, open the folder and check the images, delete cover and appendix pages that does not contain questions

# 3.OCR analyze cut to strips

In [29]:
from pytesseract import Output
import pytesseract
import pandas as pd
pytesseract.pytesseract.tesseract_cmd = r'E:\Tesseract\tesseract.exe'
from PIL import Image,ImageFont,ImageDraw, ImageFilter
import numpy as np
import random

In [30]:
from PIL import ImageDraw
def im2rects(img): 
    '''Page segmentation modes:
  0    Orientation and script detection (OSD) only.
  1    Automatic page segmentation with OSD.
  2    Automatic page segmentation, but no OSD, or OCR. (not implemented)
  3    Fully automatic page segmentation, but no OSD. (Default)
  4    Assume a single column of text of variable sizes.
  5    Assume a single uniform block of vertically aligned text.
  6    Assume a single uniform block of text.
  7    Treat the image as a single text line.
  8    Treat the image as a single word.
  9    Treat the image as a single word in a circle.
 10    Treat the image as a single character.
 11    Sparse text. Find as much text as possible in no particular order.
 12    Sparse text with OSD.
 13    Raw line. Treat the image as a single text line,
       bypassing hacks that are Tesseract-specific.'''
    custom_config = r'--psm 3'
    d=pytesseract.image_to_data(img,output_type=Output.DICT,config=custom_config)
    n_boxes = len(d['text'])

    W,H = img.size
    rectangles=[]
    min_y=H
    min_yi=0
    for i in range(n_boxes):
        if int(d['level'][i]) == 4:
            (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
            rectangles.append([(x, y), (x + w, y + h)])
            if y < min_y:
                min_y=y
                min_yi=len(rectangles)-1
            #draw.text((x, y), d['text'][i], align ="left",fill="red")
    #rectangles.pop(min_yi)
    return rectangles
    
    
def cutter(img,debug=False,precrops=[0,0,0,0],min_dy_rate=0.015,x_tol_rate=0.22):
    '''
     input: Image
     precrops: list of 4 floats between 0 and 1, left top right bottom margin rate
     output: list of cropped images
    '''
    W,H = img.size

    
    pre_left=int(W*precrops[0])
    pre_top=int(H*precrops[1])
    pre_right=int(W*(1-precrops[2]))
    pre_bottom=int(H*(1-precrops[3]))
    # precrop
    img=img.crop(tuple([pre_left,pre_top,pre_right,pre_bottom]))
    #display(img)
    W,H = img.size
    
    draw=ImageDraw.Draw(img)
    rectangles=im2rects(img)
    rectangles=sorted(rectangles,key=lambda x:x[0][0])
    if len(rectangles)==0:
        return []
    pad=5
    
    ymax=max([i[1][1] for i in rectangles])+pad
    ymin=min([i[0][1] for i in rectangles])-pad
    #xmax=max([i[1][0] for i in rectangles])+pad
    #xmin=min([i[0][0] for i in rectangles])-pad
    xmax=W
    xmin=0
    
    


    x_star=rectangles[0][0][0]
    x_tolerance=x_tol_rate*W
    cut_ys=[ymax,ymin,H,0]
    
    for rect in rectangles:
        if debug:
            draw.rectangle(rect,outline='red')
        if abs(rect[0][0]) < x_tolerance:
            cut_ys.append(rect[0][1]-pad)
    cut_ys=sorted(list(set(cut_ys)))
    # set min dy rate to prevent very small dy
    old_y=cut_ys[0]
    stable_cut_ys=[old_y]
    dy_tol=min_dy_rate*H
    for new_y in cut_ys[1:]:
        dy = new_y - old_y
        if dy > dy_tol or new_y==H:
            stable_cut_ys.append(new_y)
            old_y=new_y

    cut_ys=stable_cut_ys
        
    crop_imgs=[]
    for i,y in enumerate(cut_ys[:-1]):
        crop_imgs.append(img.crop((xmin,y,xmax,cut_ys[i+1])))
        
    if debug:
        print('rectangles:',rectangles)
        print('ymin:',ymin,'ymax',ymax)
        print('cut into',len(cut_ys)-1,', y positions:',cut_ys)
        draw.line([(0,dy_tol),(W,dy_tol)],fill ="green", width = 0)
        draw.line([(int(x_tolerance),0),(int(x_tolerance),H)],fill ="green", width = 0)
        for y in cut_ys:
            draw.line([(0,y),(W,y)],fill ='blue', width = 0)

        draw.line([(xmin,0),(xmin,H)],fill ="blue", width = 0)
        draw.line([(xmax,0),(xmax,H)],fill ="blue", width = 0)
        print('raw img:')
        display(img)
        print('cropped imgs:')
        for im in crop_imgs:
            print('-----')
            display(im)
        
    return crop_imgs


def test():
    pure_dir=os.path.join(pure_img_folder)
    fps=[p for p in get_all_files(pure_dir) if p.endswith('.png')]
    fps=random.sample(fps,5)
    fps=[r'D:\server\230620片题_物理\2-pure_questions_images\question_images\6ph04_01_que_20170616']
    for p in fps:
        print(p)
        pg=Image.open(p)
        crop_imgs=cutter(pg,debug=True)
        print(len(crop_imgs))

test()

D:\server\230620片题_物理\2-pure_questions_images\question_images\6ph04_01_que_20170616


PermissionError: [Errno 13] Permission denied: 'D:\\server\\230620片题_物理\\2-pure_questions_images\\question_images\\6ph04_01_que_20170616'

In [31]:
from PIL import Image
images_dir=pure_img_folder
strips_dir=os.path.join(task_dir,'3-images2stripes')
paper_names=os.listdir(images_dir)
paper_images_dirs=[os.path.join(images_dir,p) for p in paper_names]
#display(paper_images_dirs)

for i,paper_images_dir in tqdm(enumerate(paper_images_dirs),total=len(paper_images_dirs),leave=False):
    paper_name=paper_names[i]
    paper_strip_dir=os.path.join(strips_dir,paper_name)
    if not os.path.exists(paper_strip_dir):
        os.makedirs(paper_strip_dir)
    img_fns=sorted([fn for fn in os.listdir(paper_images_dir) if fn.endswith('.png')])
    imgs=[Image.open(os.path.join(paper_images_dir,img_fn)) for img_fn in img_fns]
    
    strips=[]
    strip_fns=[]
    cur_stripss=multi(cutter,imgs)
    for i,(img_fn,img) in enumerate(zip(img_fns,imgs)):
        #display(img)
        cur_strips=cur_stripss[i]
        #print(img_fn+' cut into',len(cur_strips))
        cur_strip_fns=[img_fn.replace('.png','_strip{:04d}.png'.format(idx+1)) for idx in range(len(cur_strips))]
        strips+=cur_strips
        strip_fns+=cur_strip_fns

    for j,s in enumerate(strips):
        with open(os.path.join(paper_strip_dir,strip_fns[j]),'wb') as f:
            s.save(f,format='PNG')
            f.close()
        

Progress: 100%|██████████| 6/6 [00:00<00:00,  9.48it/s]


# 4.extract eigentext

## get file paths

In [32]:
import threading,os
from PIL import Image

def get_file_paths(folder_path):
    file_paths = []
    for root, directories, files in os.walk(folder_path):
        for filename in files:
            file_path = os.path.join(root, filename)
            file_paths.append(file_path)
    return file_paths

def fp2txt_half(fp,half_rate=0.22,config=r'--psm 7'):
    img=Image.open(fp)
    W,H=img.size
    img=img.crop((0,0,int(W*half_rate),H))
    return pytesseract.image_to_string(img,config=config)

def fp2txt_full(fp):
    return fp2txt_half(fp,half_rate=1,config=r'--psm 3')

def test():
    src_dir=os.path.join(task_dir,'3-images2stripes')
    fps=[p for p in get_all_files(src_dir) if p.endswith('.png')]
    fps=random.sample(fps,100)
    #fps=[r'D:\server\230615test_片题2\3-images2stripes\2201ALPhysicsU1\2201ALPhysicsU1_page017_strip0009.png']
    for fp in fps:
        print(fp)
        display(Image.open(fp))
        print('text_half:',fp2txt_half(fp))
        print('text_full:',fp2txt_full(fp),'\n\n')

#test()

In [33]:
strips_dir=os.path.join(task_dir,'3-images2stripes')
all_strips_fps=get_file_paths(strips_dir)

# sanity check
for p in all_strips_fps:
    assert p.endswith('.png')

strips_excel_fp=os.path.join(task_dir,'4-strips_info.xlsx')
print('load {0} strips from {1}'.format(len(all_strips_fps),strips_dir))

st_txts_head=multi(fp2txt_half,all_strips_fps)
st_txts_full=multi(fp2txt_full,all_strips_fps)


load 115 strips from d:\server\230621片题_数学\Redo2\3-images2stripes


Progress: 100%|██████████| 115/115 [00:02<00:00, 39.09it/s]


In [34]:

st_fps=[]
st_pns=[]
st_pgns=[]
st_stns=[]

for i,fp in enumerate(all_strips_fps):

    fn=fp.split('\\')[-1]
    st_fps.append(fp)
    st_pns.append(fn.split('_page')[0])
    st_pgns.append(fn.split('_page')[-1].split('_strip')[0])
    st_stns.append(fn.split('_strip')[-1].split('.png')[0])
    img=Image.open(fp)
    #display(img)
    #st_txts.append(pytesseract.image_to_string(img,config='r--psm 7'))
    #print(text)


import pandas as pd
data=pd.DataFrame({'file_path':st_fps,
                   'paper_name':st_pns,
                   'page_num':st_pgns,
                   'strip_num':st_stns,
                   'text_head':st_txts_head,
                   'text_full':st_txts_full
                   })

data.to_excel(strips_excel_fp,index=False)


# 5-recongnize q_num

In [35]:
import pandas as pd

def qnums_valid(qnums,idxs=idxs):
    
    def is_continue(qn1,qn2):
        #print(qn1,qn2)
        lvl=len(qn1)
        if qn1==qn2:
            return True
        
        if qn1[0]=='':
            return qn2[0]==idxs[3-lvl][0]

        if qn1[0]==qn2[0]:
            if len(qn1)!=1 and len(qn2)!=1:
                return is_continue(qn1[1:],qn2[1:])
            else:
                return True
        return qn2[0] == nxt_qidx(qn1[0])
    
    def nxt_qidx(idx):
        for cur_lvl_idxs in idxs:
            if idx in cur_lvl_idxs:
                posi=cur_lvl_idxs.index(idx)
                if posi==len(cur_lvl_idxs)-1:
                    return idx
                else:
                    return cur_lvl_idxs[posi+1]
        return None
    

    r=[]
    qnums=[q.split('-') for q in qnums]
    qnums=[q+(3-len(q))*[''] for q in qnums]

    qn1=['']*3
    for qn2 in qnums:
        
        is_conti=is_continue(qn1,qn2)
        #print(qn1,qn2,is_conti)
        if not is_conti and len(r)>0:
            r[-1]=False
        r.append(is_conti)
        
        qn1=qn2
    return r

input_excel_fp=os.path.join(task_dir,'4-strips_info.xlsx')
out_excel_fp=os.path.join(task_dir,'5-recognize_qnum.xlsx')

data=pd.read_excel(input_excel_fp)

# get pre_q_num1
txts_head=list(map(str,data['text_head']))
txts_full=list(map(str,data['text_full']))
txts=[]
for head,full in zip(txts_head,txts_full):
    if head[0].isalpha():
        txts.append(full[:10])
    elif full[0].isalpha():
        txts.append(head[:10])
    else:
        txts.append(head[:7]+' '+full[:5])

pre_q_nums1=[]
print(len(txts))
lvls_idxss=[idxs[0][::-1],idxs[1],idxs[2]]
for txt in txts:
    r=[]
    txt_idxs=[]
    for lvl_idxs in lvls_idxss:
        for idx in lvl_idxs:
            if idx in txt:
                r.append(idx)
                break
    r='-'.join(r)
    pre_q_nums1.append(r)

data['pre_q_num1']=pre_q_nums1
data.to_excel(out_excel_fp,index=False)
print(out_excel_fp)


115
d:\server\230621片题_数学\Redo2\5-recognize_qnum.xlsx


In [36]:

def pre_q_num1_finish(in_fp,out_excel_fp):
    # reguliarize
    data=pd.read_excel(in_fp)
    pns=list(sorted(set(data['paper_name'])))
    datas=[data[data['paper_name']==pn] for pn in pns]
    aggregate_by_full_idx_datas=[]
    for pn,datai in zip(pns,datas):
        #datai=data[data['paper_name']==pn]
        print(pn)
        datai['pre_q_num1']=datai['pre_q_num1'].astype(str).replace('nan','')
        datai['text_full']=datai['text_full'].astype(str).replace('nan','')
        #datai['text_head']=datai['text_head'].astype(str).replace('nan','')
        pn_qnums=datai['pre_q_num1']
        pn_ftxts=datai['text_full']
        #pn_htxts=datai['text_half']
        last_valid_qn=''
        expected_cur_qn=''
        valid_qns=[]
        for cur_qn in pn_qnums:
            if '-' in cur_qn:
                cur_qn=cur_qn.split('-')[0]

            if cur_qn in idxs[0]:
                #print(cur_qn)
                if last_valid_qn =='':
                    if cur_qn==idxs[0][0]:
                        last_valid_qn=cur_qn
                        expected_cur_qn=idxs[0][idxs[0].index(last_valid_qn)+1]

                        #print('c',cur_qn)
                
                elif cur_qn != last_valid_qn and cur_qn==expected_cur_qn:
                    #print(last_valid_qn)
                    last_valid_qn=cur_qn
                    expected_cur_qn=idxs[0][idxs[0].index(last_valid_qn)+1]

            valid_qns.append(last_valid_qn)
                #print(cur_qn,expected_cur_qn,last_valid_qn)
        assert len(pn_qnums)==len(valid_qns)
        datai['valid_q_num']=valid_qns

        # append next level idxs
        full_idxs=[]
        #full_txts=[]
        cur_stack=['','','']
        #cur_txt=['','','']
        for valid_q_num, pn_qnum_whole, pn_ftxt in zip(valid_qns,pn_qnums,pn_ftxts):
            pn_qnum_lvls=pn_qnum_whole.split('-')
            for pn_qnum in pn_qnum_lvls:
                # new lvl 1 question
                if valid_q_num!=cur_stack[0]:
                    cur_stack=[valid_q_num,'','']
                else:
                    # new lvl 2 question
                    if pn_qnum in idxs[1]:
                        if pn_qnum!=cur_stack[1]:
                            cur_stack=[cur_stack[0],pn_qnum,'']
                    # new lvl 3 question
                    elif pn_qnum in idxs[2]:
                        if pn_qnum!=cur_stack[2]:
                            cur_stack=[cur_stack[0],cur_stack[1],pn_qnum]

                

            #print('valid_qnum:',valid_q_num, 'pn_qnum:',pn_qnum,'cur_stack:',cur_stack)
            full_idxs.append('-'.join([c for c in cur_stack if c !='']))
            #print(cur_txt)
            #print('\n'.join([str(c).replace('\n',' ') for c in cur_txt if c !='']))
            #full_txts.append('\n'.join([str(c).replace('\n',' ') for c in cur_txt if c !='']))
        datai['q_num_full']=full_idxs
        datai['valid']=qnums_valid(full_idxs)
        

        aggregate_by_full_idx_data=pd.DataFrame()
        agg_strip_fpss=[]
        agg_paper_names=[]
        agg_page_nums=[]
        agg_txt_full=[]
        agg_qnums=[]
        agg_full_idx=[]
        agg_valids=[]
        

        # aggregate by unique full_idx 
        change_ats=[0]
        qs_full=[datai['q_num_full'].iloc[0]]
        for i in range(1,len(datai)):
            if datai['q_num_full'].iloc[i]!= datai['q_num_full'].iloc[i-1]:
                change_ats.append(i)
                qs_full.append(datai['q_num_full'].iloc[i])
        change_ats.append(len(datai))

                
    
        for i in range(len(qs_full)):
            cur_full_idx=qs_full[i]
            cur_full_idx_data=datai.iloc[change_ats[i]:change_ats[i+1]]
            #display(cur_full_idx_data)
            if cur_full_idx =='':
                continue
                
            agg_strip_fpss.append('\n'.join([fp for fp in cur_full_idx_data['file_path']]))
            agg_paper_names.append(list(cur_full_idx_data['paper_name'])[0])
            agg_page_nums.append(list(cur_full_idx_data['page_num'])[0])
            agg_txt_full.append(' '.join([t for t in cur_full_idx_data['text_full']]).replace('\n',''))
            agg_qnums.append(str(list(cur_full_idx_data['valid_q_num'])[0]))
            agg_full_idx.append(cur_full_idx)
            agg_valids.append(all(list(cur_full_idx_data['valid'])))



            
            

        # for cur_full_idx in list(dict.fromkeys(datai['q_num_full'])):
        #     if cur_full_idx =='':
        #         continue
        #     #print(cur_full_idx)
        #     cur_full_idx_data=datai[datai['q_num_full']==cur_full_idx]

        #     agg_strip_fpss.append('\n'.join([fp for fp in cur_full_idx_data['file_path']]))
        #     agg_paper_names.append(list(cur_full_idx_data['paper_name'])[0])
        #     agg_page_nums.append(list(cur_full_idx_data['page_num'])[0])
        #     agg_txt_full.append(' '.join([t for t in cur_full_idx_data['text_full']]).replace('\n',''))
        #     agg_qnums.append(str(list(cur_full_idx_data['valid_q_num'])[0]))
        #     agg_full_idx.append(cur_full_idx)


        aggregate_by_full_idx_data['agg_strip_fpss']=agg_strip_fpss
        aggregate_by_full_idx_data['agg_paper_names']=agg_paper_names
        aggregate_by_full_idx_data['agg_page_nums']=agg_page_nums
        aggregate_by_full_idx_data['agg_txt_full']=agg_txt_full
        aggregate_by_full_idx_data['agg_qnum']=agg_qnums
        aggregate_by_full_idx_data['agg_full_qnum']=agg_full_idx
        aggregate_by_full_idx_data['agg_valids']=agg_valids

        aggregate_by_full_idx_datas.append(aggregate_by_full_idx_data)

    with pd.ExcelWriter(out_excel_fp) as writer:
        pd.concat(datas).to_excel(writer,sheet_name='raw',index=False)
        pd.concat(aggregate_by_full_idx_datas).to_excel(writer,sheet_name='agg_by_full_qnum',index=False)


In [37]:

pre_q_num1_finish(in_fp=out_excel_fp,out_excel_fp=out_excel_fp)

wdm11_01_que_20230126


# 6-sanity check

In [38]:
import shutil
step_dir_6=os.path.join(task_dir,'6-sanity_checkpoint')
if not os.path.exists(step_dir_6):
    os.mkdir(step_dir_6)
src=os.path.join(task_dir,'5-recognize_qnum.xlsx')
dst=os.path.join(task_dir,step_dir_6,'sanity_check.xlsx')
shutil.copy(src,dst)

'd:\\server\\230621片题_数学\\Redo2\\6-sanity_checkpoint\\sanity_check.xlsx'

In [39]:
from PIL import Image
import os
import pandas as pd

def concatenate_images_vertically(file_paths):
    images = [Image.open(file_path) for file_path in file_paths]
    widths, heights = zip(*(i.size for i in images))

    total_width = max(widths)
    total_height = sum(heights)

    concatenated_image = Image.new('RGB', (total_width, total_height))

    y_offset = 0
    for image in images:
        concatenated_image.paste(image, (0, y_offset))
        y_offset += image.height

    return concatenated_image


def fps2imgs(fps):
    #if len(fps)>2 and fps[-1].split('page')[-1].split('_')[0]!=fps[-2].split('page')[-2].split('_')[0]:
    #    fps=fps[:-1]

    fpss=[]
    old_idx=0
    old_pgn=fps[0].split('page')[-1].split('_')[0]
    for i,p in enumerate(fps):
        new_pgn=p.split('page')[-1].split('_')[0]
        #print(old_pgn,new_pgn)
        if new_pgn != old_pgn:
            fpss.append(fps[old_idx:i])
            old_idx=i
            old_pgn=new_pgn
    if old_idx<len(fps)-1:
        fpss.append(fps[old_idx:])
    
    imgs=multi(concatenate_images_vertically,fpss)
    #for fs in fpss:
    #    imgs.append(concatenate_images_vertically(fs))
    
    return imgs

def pn2q_imgs(pn,out_dir=os.path.join(task_dir,'6-sanity_checkpoint')):
    #print(pn)
    pn_out_dir=os.path.join(out_dir,pn)
    if not os.path.exists(pn_out_dir):
        os.makedirs(pn_out_dir)
    pn_data=data[data['agg_paper_names']==pn]


    for k in ['agg_strip_fpss','agg_txt_full','agg_valids']:
        #print(k)
        #display(pn_data)
        if k == 'agg_valids':
            pn_data[k] = pn_data[[k,'agg_qnum']].groupby(['agg_qnum'])[k].transform(lambda x: all([xx=='True' for xx in x]))
            continue
        pn_data[k] = pn_data[[k,'agg_qnum']].groupby(['agg_qnum'])[k].transform(lambda x: '\n'.join(x))
        

    #display(pn_data)
    df2=pn_data[['agg_strip_fpss','agg_txt_full','agg_qnum','agg_paper_names','agg_valids']].drop_duplicates()
    #display(df2)
    #df2.to_excel(os.path.join(out_dir,'mm.xlsx'),index=False)

    q_imgs_fpss=[]
    for i in range(len(df2)):
        q_num=df2['agg_qnum'].iloc[i]
        fps=df2['agg_strip_fpss'].iloc[i].split('\n')
        #for p in fps:
            #print(p)

        imgs=fps2imgs(fps)
        q_imgs_fps=[]
        for pic_i,im in enumerate(imgs):
            fn='Q{:03d}-pic{:02d}.png'.format(i+1,pic_i+1)
            fp=os.path.join(pn_out_dir,fn)
            im.save(fp)
            q_imgs_fps.append(fp)
        q_imgs_fpss.append('\n'.join(q_imgs_fps))
    df2['q_imgs_fpss']=q_imgs_fpss

    df2.to_excel(os.path.join(pn_out_dir,pn+'.xlsx'),index=False)

    return all(list(df2['agg_valids']))

def test():
    input_excel_fp=os.path.join(task_dir,'5-recognize_qnum.xlsx')
    data=pd.read_excel(input_excel_fp,sheet_name='agg_by_full_qnum',dtype=str).astype(str)
    data['agg_full_qnum']=data['agg_full_qnum'].astype(str)
    pns=list(dict.fromkeys(data['agg_paper_names']))
    multi(pn2q_imgs,pns[:2])


#test()

In [40]:
in_fp=os.path.join(step_dir_6,'sanity_check.xlsx')
pre_q_num1_finish(in_fp=in_fp,out_excel_fp=in_fp)


data=pd.read_excel(in_fp,sheet_name='agg_by_full_qnum',dtype=str).astype(str)
data['agg_full_qnum']=data['agg_full_qnum'].astype(str)
pns=list(dict.fromkeys(data['agg_paper_names']))
multi(pn2q_imgs,pns)

# copy strips
for pn in pns:
    src=os.path.join(task_dir,'3-images2stripes',pn)
    dst=os.path.join(step_dir_6,pn,'strips')
    shutil.copytree(src,dst)

wdm11_01_que_20230126


Progress: 100%|██████████| 1/1 [00:00<00:00,  2.48it/s]


## Now, open the '6-sanity_check.xlsx' and manually check&fix the pre_qnum1 column

In [41]:
import pandas as pd
step_dir_6=os.path.join(task_dir,'6-sanity_checkpoint')
def delete_rows_by_names(names, file_path, column_name, out_path=None):
    if out_path == None:
        out_path=file_path
    # Read the Excel file into a pandas DataFrame
    df = pd.read_excel(file_path)
    
    # Filter out the rows where the given column's value is in the list of names
    df = df[~df[column_name].isin(names)]
    
    # Save the modified DataFrame back to the Excel file
    df.to_excel(out_path, index=False)

bad_pns='''d1 2020 1月 问卷'''.split('\n')
display(bad_pns)

white_fp=os.path.join(step_dir_6,'sanity_check_white.xlsx')
delete_rows_by_names(bad_pns,
                     os.path.join(step_dir_6,'sanity_check.xlsx'),
                     'paper_name',
                     out_path=white_fp)


# move bad paper into Redo
import shutil
os.makedirs(os.path.join(task_dir,'Redo','0-input'))
rd=0
for fp in get_all_files(os.path.join(task_dir,'0-input')):
    #print(fp)
    for pn in bad_pns:
        if pn+'.pdf' == fp.split('\\')[-1]:
            shutil.copy(fp,os.path.join(task_dir,'Redo','0-input',fp.split('\\')[-1]))
            rd+=1
            break
assert rd==len(bad_pns)

['d1 2020 1月 问卷']

AssertionError: 

# 7-concate image & output as task1

In [42]:
step_dir_6=os.path.join(task_dir,'6-sanity_checkpoint')
step_dir_7=os.path.join(task_dir,'7-concat_image')
def pn2q_imgs2(pn,out_dir=step_dir_7):
    return pn2q_imgs(pn,out_dir=out_dir)

In [43]:

if not os.path.exists(step_dir_7):
    os.mkdir(step_dir_7)

in_fp=os.path.join(step_dir_6,'sanity_check.xlsx')
out_fp=os.path.join(step_dir_7,'concate_image.xlsx')
pre_q_num1_finish(in_fp=in_fp,out_excel_fp=out_fp)

data=pd.read_excel(out_fp,sheet_name='agg_by_full_qnum',dtype=str).astype(str)
data['agg_full_qnum']=data['agg_full_qnum'].astype(str)
pns=list(dict.fromkeys(data['agg_paper_names']))
multi(pn2q_imgs2,pns)


wdm11_01_que_20230126


Progress: 100%|██████████| 1/1 [00:00<00:00,  2.51it/s]


[True]

In [44]:
# convert to task1
def fp2fp(in_fp,out_fp):
    data=pd.read_excel(in_fp)
    data_frq=data[data['Ques_type']=='FRQ']
    data_mcq=data[data['Ques_type']=='MCQ']


    txts,q_nums,p_names=list(data_frq['Ques_text']),list(data_frq['Ques_number']),list(data_frq['Paper_name'])

    q_txts=txts
    out_frq_data=pd.DataFrame(columns=list(data.keys()))
    q_num2txt=dict()

    q_n=[]
    q_tx=[]
    q_pn=[]
    q_type=[]

    for i,t in enumerate(q_txts):
        q_num=str(q_nums[i])
        #print(t)
        t=str(t)
        if len(t)==0 or t=='nan':
            q_n.append(q_num)
            q_tx.append('')
            q_pn.append(p_names[i])
            q_type.append('FRQ')
        qs=break_q(t)
        if qs['context']!='can not break':
            #print(q_num+'-context: '+qs['context'])
            for k in qs:
                if k != 'original' and k != 'context':
                    qs[k]=break_q(qs[k],idx_pointers=['(i)','(ii)','(iii)','(iv)','(v)'])
                    if qs[k]['context']!='can not break':
                        #print(q_num+'-'+k+'-context: '+qs[k]['context'])
                        for kk in qs[k]:
                            if kk != 'original' and kk!='context':
                                q_idx=q_num+'-'+k+'-'+kk
                                t=qs['context']+qs[k]['context']+qs[k][kk]
                                #print(q_idx+': '+t)
                                q_num2txt[q_idx]=t
                                q_n.append(q_idx)
                                q_tx.append(t)
                                q_pn.append(p_names[i])
                                q_type.append('FRQ')
                    else:
                        q_idx=q_num+'-'+k
                        t=qs['context']+qs[k]['original']
                        #print(q_idx+': '+t)
                        q_num2txt[q_idx]=t
                        q_n.append(q_idx)
                        q_tx.append(t)
                        q_pn.append(p_names[i])
                        q_type.append('FRQ')
                    

        else:
            q_idx=q_num
            t=qs['original']
            #print(q_idx+': '+t)
            q_num2txt[q_idx]=t
            q_n.append(q_idx)
            q_tx.append(t)
            q_pn.append(p_names[i])
            q_type.append('FRQ')

    out_frq_data['Ques_number']=q_n
    out_frq_data['Ques_text']=q_tx
    out_frq_data['Paper_name']=q_pn
    out_frq_data['Ques_type']=q_type


    out_data=pd.concat([data_mcq,out_frq_data])
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter(out_fp)

    # Write each dataframe to a different worksheet.
    out_data.to_excel(writer, sheet_name="content",index=False)
    head_des=pd.read_excel(r"D:\BaiduSyncdisk\广州外校\领启\230512第一阶段\tool_scripts\0610split_small_question\input\questions_extracted_AL-Chemistry-U1.xlsx",sheet_name='head_description')
    head_des.to_excel(writer, sheet_name="head_description",index=False)
    # Close the Pandas Excel writer and output the Excel file.
    writer.close()
    #print(out_data)

    
def break_q(que,idx_pointers=['({0})'.format(idx) for idx in ['a','b','c','d','e','f','g','h']]):
    qs=dict({'original':que})
    #print(que)
    # extract context
    cut_idxs=[que.find(idx) for idx in idx_pointers]
    if cut_idxs[0]==-1:
        qs['context']='can not break'
        return qs
    for i in range(len(cut_idxs)-1):
        if cut_idxs[i]==-1:
            break
        if i==0:
            qs['context']=que[:cut_idxs[0]]
        qs[idx_pointers[i]]=que[cut_idxs[i]:cut_idxs[i+1]]
        #print(idx_pointers[i],que[cut_idxs[i]:cut_idxs[i+1]])
    return qs


def excel_fp2task1_excel(in_fp,debug=False):
    fn=in_fp.split('\\')[-1]
    print(fn)
    data=pd.read_excel(in_fp)
    if debug:
        display(data)
        
    data_new=pd.DataFrame(columns=['Ques_topic_by_men','Ques_topic_by_GPT',
                                   'Similarity','Ques_text','Paper_name',
                                   'Ques_number','Ques_type'])
    
    data_new['Ques_text']=data['agg_txt_full']
    data_new['Paper_name']=[fn.replace('.xlsx','')]*len(data)
    data_new['Ques_number']=data['agg_qnum']
    data_new['Ques_type']=['FRQ']*len(data)
    if debug:
        display(data_new)
    if not os.path.exists(os.path.join(step_dir_7,'task1')):
        os.mkdir(os.path.join(step_dir_7,'task1'))

    out_ex_fp=os.path.join(step_dir_7,'task1','question_extracted_'+fn)
    data_new.to_excel(out_ex_fp,index=False)
    fp2fp(out_ex_fp,out_ex_fp)
    return 0





def test():
    fps=[fp for fp in get_all_files(step_dir_7) if fp.endswith('.xlsx') and 'concate_image' not in fp]
    display(fps)
    excel_fp2task1_excel(fps[3],debug=True)

#test()

In [45]:
fps=[fp for fp in get_all_files(step_dir_7) if fp.endswith('.xlsx') and 'concate_image' not in fp]
multi(excel_fp2task1_excel,fps)

concate_image.xlsx
wdm11_01_que_20230126.xlsx


Progress:   0%|          | 0/2 [00:00<?, ?it/s]Exception in thread Thread-905:
Traceback (most recent call last):
  File "c:\Users\hts\anaconda3\envs\Productivity\lib\site-packages\pandas\core\indexes\base.py", line 3080, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 70, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 101, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 4554, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 4562, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'agg_txt_full'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\hts\anaconda3\envs\Productivity\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "c:\Users\hts\anaconda3\envs\Productivity\lib\threading.py", line 917, in r

[None, 0]